In [32]:
import pandas as pd
import json
import requests
from socketIO_client import SocketIO

In [33]:
TRADING_API_URL = 'https://api-demo.fxcm.com:443'
WEBSOCKET_PORT = 443
ACCESS_TOKEN = 'a2a3a4cf0621891c59e51d4d8ccf6c410b82da3d'

### Connect

In [34]:
def on_connect():
    print('Websocket Connected: ' + socketIO._engineIO_session.id)

def on_close():
        print('Websocket Closed.')

socketIO =  SocketIO(TRADING_API_URL, WEBSOCKET_PORT, params={'access_token' : ACCESS_TOKEN})

socketIO.on('connect', on_connect)
socketIO.on('disconnect', on_close)

bearer_access_token = "Bearer " + socketIO._engineIO_session.id + ACCESS_TOKEN

print(bearer_access_token)

Bearer 6IcATp2fupFiXMSnAAEYa2a3a4cf0621891c59e51d4d8ccf6c410b82da3d


### Pull historical prices

In [35]:
method = '/candles/1/H1'

hist_response = requests.get(TRADING_API_URL + method,
                             headers = {
                                 'User-Agent': 'request',
                                 'Authorization': bearer_access_token,
                                 'Accept': 'application/json',
                                 'Content-Type': 'application/x-wwww-form-urlencoded'
                             },
                             params = {
                                 'num': 1000,
                                 'from': 1494086400,
                                 'to': 1503835200
                             })

if hist_response.status_code == 200:
  print ("Data retrieved...")

  hist_data = hist_response.json()
  candle_data = hist_data['candles']

  df = pd.DataFrame(candle_data)
  df.columns = ["time", "bidopen", "bidclose", "bidhigh", "bidlow", "askopen", "askclose", "askhigh", "asklow", "TickQty"]
  df['time'] = pd.to_datetime(df['time'], unit='s')
  print(df)
else:
  print(hist_response)


Data retrieved...
                    time  bidopen  bidclose  bidhigh   bidlow  askopen  \
0    2017-05-07 19:00:00  1.09978   1.10184  1.10214  1.10151  1.10010   
1    2017-05-07 20:00:00  1.10184   1.10161  1.10221  1.10139  1.10207   
2    2017-05-07 21:00:00  1.10161   1.10111  1.10209  1.10010  1.10228   
3    2017-05-07 22:00:00  1.10111   1.09927  1.10132  1.09845  1.10143   
4    2017-05-07 23:00:00  1.09927   1.09644  1.09956  1.09631  1.09937   
5    2017-05-08 00:00:00  1.09644   1.09665  1.09739  1.09566  1.09654   
6    2017-05-08 01:00:00  1.09665   1.09744  1.09761  1.09631  1.09673   
7    2017-05-08 02:00:00  1.09744   1.09750  1.09823  1.09713  1.09750   
8    2017-05-08 03:00:00  1.09750   1.09710  1.09755  1.09652  1.09757   
9    2017-05-08 04:00:00  1.09710   1.09720  1.09726  1.09607  1.09718   
10   2017-05-08 05:00:00  1.09720   1.09937  1.09944  1.09704  1.09728   
11   2017-05-08 06:00:00  1.09937   1.09835  1.09964  1.09736  1.09949   
12   2017-05-08 07:0

### Subscribe for one update 

In [36]:
method = '/subscribe'

sub_response = requests.post(TRADING_API_URL + method,
                           headers = {
                                 'User-Agent': 'request',
                                 'Authorization': bearer_access_token,
                                 'Accept': 'application/json',
                                 'Content-Type': 'application/x-www-form-urlencoded'
                             },
                             data = {
                                 'pairs': 'AUD/USD'
                             })

print(sub_response)
print(sub_response.json())

<Response [200]>
{'response': {'executed': True, 'error': ''}, 'pairs': [{'Updated': 1550184405536, 'Rates': [0.71024, 0.71041, 0.71085, 0.71018, 0.71024, 0.71041], 'Symbol': 'AUD/USD'}]}


### Subscribe for streaming update 

In [37]:
def on_price_update(msg):
    response = json.loads(msg)
    print(response)

if sub_response.status_code == 200:
    socketIO.on('AUD/USD', on_price_update)
    socketIO.wait()

Websocket Connected: 6IcATp2fupFiXMSnAAEY
{'Updated': 1550184413845, 'Rates': [0.71024, 0.71041, 0.71085, 0.71018], 'Symbol': 'AUD/USD'}
{'Updated': 1550184414536, 'Rates': [0.71025, 0.71042, 0.71085, 0.71018], 'Symbol': 'AUD/USD'}
{'Updated': 1550184414833, 'Rates': [0.71023, 0.7104, 0.71085, 0.71018], 'Symbol': 'AUD/USD'}
{'Updated': 1550184420821, 'Rates': [0.71024, 0.71041, 0.71085, 0.71018], 'Symbol': 'AUD/USD'}
{'Updated': 1550184422822, 'Rates': [0.71023, 0.7104, 0.71085, 0.71018], 'Symbol': 'AUD/USD'}
{'Updated': 1550184460561, 'Rates': [0.71023, 0.7104, 0.71085, 0.71018], 'Symbol': 'AUD/USD'}
{'Updated': 1550184462930, 'Rates': [0.71023, 0.7103900000000002, 0.71085, 0.71018], 'Symbol': 'AUD/USD'}
{'Updated': 1550184463212, 'Rates': [0.71024, 0.71041, 0.71085, 0.71018], 'Symbol': 'AUD/USD'}
{'Updated': 1550184469103, 'Rates': [0.71027, 0.71041, 0.71085, 0.71018], 'Symbol': 'AUD/USD'}
{'Updated': 1550184469288, 'Rates': [0.71025, 0.71042, 0.71085, 0.71018], 'Symbol': 'AUD/USD'}


KeyboardInterrupt: 